In [ ]:
# ============================
# BLOQUE 1 — Descarga y preparación de datos
# ============================

import yfinance as yf
import pandas as pd
import numpy as np

# ============================
# Caso 1 — Año 2024 (3 trimestres entrenamiento, 1 mes prueba)
# ============================

fecha_inicial_2024 = pd.Timestamp('2024-01-01')
fecha_final_2024   = pd.Timestamp('2024-12-31')

# Entrenamiento: 1 enero → 30 septiembre
fecha_fin_entrenamiento_2024 = pd.Timestamp('2024-09-30')

# Prueba: 1 octubre → 31 diciembre
fecha_inicio_prueba_2024     = pd.Timestamp('2024-10-01')

# Descarga de datos diarios (solo año 2024)
datos_2024 = yf.download(
    tickers='BTC-USD',
    start=fecha_inicial_2024.strftime('%Y-%m-%d'),
    end=fecha_final_2024.strftime('%Y-%m-%d'),
    interval='1d',
    auto_adjust=False
)

# Limpieza
datos_2024 = datos_2024.reset_index()
datos_2024 = datos_2024.rename(columns=str)
datos_2024 = datos_2024[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].dropna()
datos_2024['Date'] = pd.to_datetime(datos_2024['Date'])
datos_2024 = datos_2024.sort_values('Date').reset_index(drop=True)

# Etiquetas para el split
datos_2024['Split'] = np.where(
    datos_2024['Date'] <= fecha_fin_entrenamiento_2024,
    'train_2024_9m',
    'test_2024_3m'
)

# ============================
# Caso 2 — Desde 2010 hasta final de 2024
# ============================

fecha_inicio_hist = pd.Timestamp('2010-07-01')
fecha_fin_hist    = pd.Timestamp('2024-12-31')

# Descarga
datos_total = yf.download(
    tickers='BTC-USD',
    start=fecha_inicio_hist.strftime('%Y-%m-%d'),
    end=fecha_fin_hist.strftime('%Y-%m-%d'),
    interval='1d',
    auto_adjust=False
)

# Limpieza
datos_total = datos_total.reset_index()
datos_total = datos_total.rename(columns=str)
datos_total = datos_total[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].dropna()
datos_total['Date'] = pd.to_datetime(datos_total['Date'])
datos_total = datos_total.sort_values('Date').reset_index(drop=True)

# División: historial completo para entrenar, 2024 para prueba
fecha_inicio_test_hist = pd.Timestamp('2024-01-01')

datos_total['Split'] = np.where(
    datos_total['Date'] < fecha_inicio_test_hist,
    'train_full_history_until_2023',
    'test_2024_full_history'
)

# Guardado
datos_total.to_csv("BTC_2010_2024_Yahoo_Limpio.csv", index=False)

# ============================
# Resumen visual
# ============================

print("========================================")
print("RESUMEN — Caso 1: Año 2024")
print("========================================")
print(datos_2024.head(3))
print(datos_2024.tail(3))
print(f"Rango total: {datos_2024['Date'].min().date()} -> {datos_2024['Date'].max().date()}")
print(f"Entrenamiento (9 meses): {len(datos_2024[datos_2024['Split']=='train_2024_9m'])} filas")
print(f"Prueba (3 meses): {len(datos_2024[datos_2024['Split']=='test_2024_3m'])} filas\n")

print("===========================================")
print("RESUMEN — Caso 2: Historia completa → Test en 2024")
print("===========================================")
print(datos_total.head(3))
print(datos_total.tail(3))
print(f"Entrenamiento: 2010 → 2023 | Filas: {len(datos_total[datos_total['Split']=='train_full_history_until_2023'])}")
print(f"Prueba: 2024 | Filas: {len(datos_total[datos_total['Split']=='test_2024_full_history'])}")


In [ ]:
# ============================
# BLOQUE 2 — Escalado, secuencias y utilidades Walk-Forward
# ============================

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------------------
# Ventana e horizontes
# ---------------------------------------

WINDOW = 60
H_SEMANAL = 7
H_MENSUAL = 30

# ---------------------------------------
# Preparar arrays multivariados y fechas
# ---------------------------------------

cols_features = ['Open', 'High', 'Low', 'Close', 'Volume']

data_2024_raw = datos_2024[cols_features].values.astype(float)
dates_2024 = datos_2024['Date'].values

data_total_raw = datos_total[cols_features].values.astype(float)
dates_total = datos_total['Date'].values


# ---------------------------------------
# Función para crear secuencias
# ---------------------------------------

def crear_secuencias_multivariadas(data_scaled, window, horizon, target_col=3):
    X, y = [], []
    n = len(data_scaled)
    for i in range(n - window - horizon + 1):
        X.append(data_scaled[i : i + window, :])
        y.append(data_scaled[i + window + horizon - 1, target_col])
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    return X, y


# ---------------------------------------
# Escaladores independientes
# ---------------------------------------

def scale_using_train_only(data_raw, dates, train_end_date):
    mask_train_rows = dates <= np.datetime64(train_end_date)
    train_cut_idx = np.where(mask_train_rows)[0].max()

    scaler = MinMaxScaler()
    scaler.fit(data_raw[: train_cut_idx + 1, :])  # Fit solo en entrenamiento
    data_scaled = scaler.transform(data_raw)

    return scaler, data_scaled, train_cut_idx


# ======================================================
# Definición del Walk-Forward
# ======================================================

def walk_forward_generator(X_all, y_all, fechas_objetivo, mask_test_seq):
    test_indices = np.where(mask_test_seq)[0]
    for idx in test_indices:

        if idx == 0:
            X_train_cur = np.zeros((0, X_all.shape[1], X_all.shape[2]), dtype=X_all.dtype)
            y_train_cur = np.zeros((0, 1), dtype=y_all.dtype)
        else:
            X_train_cur = X_all[:idx]
            y_train_cur = y_all[:idx]

        X_pred = X_all[idx:idx+1]
        y_true = y_all[idx]
        fecha_obj = fechas_objetivo[idx]

        yield X_train_cur, y_train_cur, X_pred, y_true, fecha_obj, idx

# ---------------------------------------
# CASO 1: 2024
# ---------------------------------------

scaler_2024, serie_2024_scaled, train_cut_idx_2024 = scale_using_train_only(
    data_2024_raw, dates_2024, fecha_fin_entrenamiento_2024
)

X_2024, y_2024 = crear_secuencias_multivariadas(
    serie_2024_scaled, WINDOW, H_SEMANAL, target_col=3
)

fechas_objetivo_2024 = dates_2024[WINDOW + H_SEMANAL - 1 : ]

mask_train_seq_2024 = fechas_objetivo_2024 <= np.datetime64(fecha_fin_entrenamiento_2024)
mask_test_seq_2024  = fechas_objetivo_2024 >= np.datetime64(fecha_inicio_prueba_2024)

X_train_2024 = X_2024[mask_train_seq_2024]
y_train_2024 = y_2024[mask_train_seq_2024]
X_test_2024  = X_2024[mask_test_seq_2024]
y_test_2024  = y_2024[mask_test_seq_2024]

# Crear Walk-Forward
walk_2024 = list(walk_forward_generator(
    X_2024, y_2024, fechas_objetivo_2024, mask_test_seq_2024
))


# ---------------------------------------
# CASO 2: Histórico
# ---------------------------------------

scaler_total, serie_total_scaled, train_cut_idx_total = scale_using_train_only(
    data_total_raw, dates_total, fecha_inicio_test_hist - np.timedelta64(1, 'D')
)

X_total, y_total = crear_secuencias_multivariadas(
    serie_total_scaled, WINDOW, H_MENSUAL, target_col=3
)

fechas_objetivo_total = dates_total[WINDOW + H_MENSUAL - 1 : ]

mask_test_seq_total = fechas_objetivo_total >= np.datetime64(fecha_inicio_test_hist)

X_train_total = X_total[~mask_test_seq_total]
y_train_total = y_total[~mask_test_seq_total]
X_test_total  = X_total[mask_test_seq_total]
y_test_total  = y_total[mask_test_seq_total]

# Crear Walk-Forward
walk_total = list(walk_forward_generator(
    X_total, y_total, fechas_objetivo_total, mask_test_seq_total
))


# ---------------------------------------
# Resumen shapes
# ---------------------------------------

def resumen_shapes():
    print("=== RESUMEN SECUENCIAS ===")
    print("-- 2024 --")
    print("Raw rows:", data_2024_raw.shape[0], " -> sequences:", X_2024.shape[0])
    print("Train seqs:", X_train_2024.shape[0], " Test seqs:", X_test_2024.shape[0])
    print("")
    print("-- HISTÓRICO --")
    print("Raw rows:", data_total_raw.shape[0], " -> sequences:", X_total.shape[0])
    print("Train seqs:", X_train_total.shape[0], " Test seqs:", X_test_total.shape[0])
    print("")
    print("Scaler 2024 fitted en filas 0..", train_cut_idx_2024)
    print("Scaler total fitted en filas 0..", train_cut_idx_total)


# Ejecutar resumen
resumen_shapes()

In [ ]:
# ============================
# BLOQUE 3 — LSTM Walk-Forward Paralelizado
# ============================

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import random
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DISPOSITIVO ACTIVO:", device)

# ------------------------------
# Fijar semillas
# ------------------------------

def fijar_semillas(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True


# ------------------------------
# Modelo LSTM
# ------------------------------

class LSTMModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=64, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])


# ------------------------------
# ENTRENAR UNA SOLA VENTANA
# ------------------------------

def entrenar_lstm_un_paso(Xtr, ytr, epochs=30, lr=1e-3, batch_size=64):

    ds = TensorDataset(
        torch.tensor(Xtr, dtype=torch.float32),
        torch.tensor(ytr, dtype=torch.float32).view(-1, 1)
    )
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

    model = LSTMModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for _ in range(epochs):
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)

            pred = model(xb)
            loss = criterion(pred, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model


# ------------------------------
# Walk-Forward
# ------------------------------

def predecir_walk_forward_lstm(generator, repeticiones=1, epochs=30, lr=1e-3):

    predicciones = []

    for (Xtr, ytr, Xfuture, ytrue, fecha_objetivo, idx) in tqdm(generator,
                                                               desc="Walk-Forward LSTM (rápido)"):

        preds_rep = []

        for r in range(repeticiones):
            fijar_semillas(100 + r)

            model = entrenar_lstm_un_paso(Xtr, ytr,
                                          epochs=epochs,
                                          lr=lr,
                                          batch_size=64)  # <<< más rápido

            model.eval()

            Xf = torch.tensor(Xfuture, dtype=torch.float32).to(device)
            with torch.no_grad():
                pred = model(Xf).cpu().numpy().flatten()[0]

            preds_rep.append(pred)

        predicciones.append({
            "fecha": fecha_objetivo,
            "pred": np.mean(preds_rep),
            "pred_std": np.std(preds_rep),
            "index": idx
        })

    return predicciones


# ------------------------------
# EJECUCIÓN
# ------------------------------

print("\nEjecutando LSTM con Walk-Forward (rápido)...")

preds_walk_2024 = predecir_walk_forward_lstm(
    walk_2024, repeticiones=5, epochs=16, lr=1e-3
)

preds_walk_total = predecir_walk_forward_lstm(
    walk_total, repeticiones=5, epochs=16, lr=1e-3
  )
print("LSTM con Walk-Forward completado.")

In [ ]:
# ============================
# BLOQUE 4 — Transformer Walk-Forward Paralelizado
# ============================

import torch
import torch.nn as nn
import numpy as np
import random
from tqdm import tqdm
import math

# ------------------------------
# Dispositivo
# ------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DISPOSITIVO ACTIVO:", device)

# ------------------------------
# Fijar semillas
# ------------------------------

def fijar_semillas(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True      # <--- aceleración
    torch.backends.cudnn.deterministic = False


# ------------------------------
# Positional Encoding
# ------------------------------

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        return self.pe[:, :x.size(1), :]


# ------------------------------
# Modelo Transformer
# ------------------------------

class TransformerModel(nn.Module):
    def __init__(self, seq_len=60, d_model=64, nhead=4,
                 num_layers=2, dim_feedforward=256, dropout=0.1):
        super().__init__()

        self.input_proj = nn.Linear(5, d_model)
        self.positional = PositionalEncoding(d_model, max_len=seq_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True,
            dropout=dropout,
            activation="gelu"
        )

        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_proj(x) + self.positional(x)
        x = self.encoder(x)
        x = self.norm(x)
        return self.head(x[:, -1, :])


# ------------------------------
# ENTRENAR UNA VENTANA
# ------------------------------

def entrenar_transformer_un_paso(X_train, y_train,
                                 epochs=40, lr=1e-4, batch_size=64):

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

    ds = torch.utils.data.TensorDataset(X_train, y_train)
    dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True)

    model = TransformerModel().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for _ in range(epochs):
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    return model


# ------------------------------
# WALK-FORWARD
# ------------------------------

def predecir_walk_forward_transformer(generator,
                                      repeticiones=1, epochs=40, lr=1e-4):

    predicciones = []

    for (Xtr, ytr, Xfuture, ytrue, fecha_objetivo, idx) in tqdm(generator,
                                                                desc="Walk-Forward Transformer (rápido)"):

        preds_rep = []
        Xfuture_tensor = torch.tensor(Xfuture, dtype=torch.float32).to(device)

        for r in range(repeticiones):
            fijar_semillas(300 + r)

            model = entrenar_transformer_un_paso(
                Xtr, ytr, epochs=epochs, lr=lr, batch_size=64  # <--- más rápido
            )
            model.eval()

            with torch.no_grad():
                pred = model(Xfuture_tensor).cpu().numpy().flatten()[0]
                preds_rep.append(pred)

        predicciones.append({
            "fecha": fecha_objetivo,
            "pred": np.mean(preds_rep),
            "pred_std": np.std(preds_rep),
            "index": idx
        })

    return predicciones


# ------------------------------
# EJECUCIÓN
# ------------------------------

print("\nEjecutando Transformer con Walk-Forward...")

preds_walk_2024_tf = predecir_walk_forward_transformer(
    walk_2024, repeticiones=5, epochs=16, lr=1e-4
)

preds_walk_total_tf = predecir_walk_forward_transformer(
    walk_total, repeticiones=5, epochs=16, lr=1e-4
)

print("Transformer con Walk-Forward completado.")

In [ ]:
# ============================
# BLOQUE 5 — Evaluación y visualización
# ============================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
import pandas as pd

EPS = 1e-8

# ------------------------------
# Utilidades
# ------------------------------

def ensure_1d(arr):
    a = np.asarray(arr)
    if a.ndim == 2 and a.shape[1] == 1:
        return a.ravel()
    return a.reshape(-1)

def desescalar(scaler, arr_scaled):
    a = np.asarray(arr_scaled).reshape(-1, 1)
    dummy = np.zeros((len(a), 5))
    dummy[:, 3] = a[:, 0]  # 3 = columna CLOSE dentro del MinMax multivariante
    return scaler.inverse_transform(dummy)[:, 3]

def align_lengths(y, *preds):
    """
    Recorta todos los arrays al largo mínimo (usualmente usar y como referencia)
    Retorna (y_trim, pred1_trim, pred2_trim, ...)
    """
    L = min(len(y), *(len(p) for p in preds))
    y_t = np.asarray(y)[:L]
    preds_t = [np.asarray(p)[:L] for p in preds]
    return (y_t, *preds_t)

# ------------------------------
# Extraer predicciones de los walks (listas de dicts)
# ------------------------------

preds_2024 = np.array([p["pred"] for p in preds_walk_2024])
preds_2024_tf = np.array([p["pred"] for p in preds_walk_2024_tf])

preds_total = np.array([p["pred"] for p in preds_walk_total])
preds_total_tf = np.array([p["pred"] for p in preds_walk_total_tf])

# Asegurar forma correcta (provenientes de Bloque 2)
y_test_2024 = ensure_1d(y_test_2024)
y_test_total = ensure_1d(y_test_total)

# ------------------------------
# Desescalar (usar scalers de Bloque 2)
# ------------------------------

y_test_2024_real   = desescalar(scaler_2024, y_test_2024)
preds_2024_real    = desescalar(scaler_2024, preds_2024)
preds_2024_tf_real = desescalar(scaler_2024, preds_2024_tf)

y_test_total_real   = desescalar(scaler_total, y_test_total)
preds_total_real    = desescalar(scaler_total, preds_total)
preds_total_tf_real = desescalar(scaler_total, preds_total_tf)

# ------------------------------
# Alinear longitudes por seguridad
# ------------------------------

y_test_2024_real, preds_2024_real, preds_2024_tf_real = align_lengths(
    y_test_2024_real, preds_2024_real, preds_2024_tf_real
)

y_test_total_real, preds_total_real, preds_total_tf_real = align_lengths(
    y_test_total_real, preds_total_real, preds_total_tf_real
)

# ------------------------------
# Métricas de evaluación
# ------------------------------

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), EPS, None))) * 100

def smape(y_true, y_pred):
    return np.mean(2 * np.abs(y_true - y_pred) /
                   (np.abs(y_true) + np.abs(y_pred) + EPS)) * 100

def precision_promedio(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    return 100 * (1 - mae / (np.mean(y_true) + EPS))

def precision_punto_a_punto(y_true, y_pred):
    return np.mean(100 * (1 - np.abs(y_true - y_pred) / (np.abs(y_true) + EPS)))

def directional_accuracy(y_true, y_pred):
    if len(y_true) < 2 or len(y_pred) < 2:
        return np.nan
    return np.mean(np.sign(np.diff(y_true)) == np.sign(np.diff(y_pred))) * 100

def pearson_safe(y_true, y_pred):
    try:
        if len(y_true) < 2 or len(y_pred) < 2:
            return np.nan
        return pearsonr(y_true, y_pred)[0]
    except Exception:
        return np.nan

def resumen_metrics(y_true, y_pred, nombre):
    mae = mean_absolute_error(y_true, y_pred)
    r = rmse(y_true, y_pred)
    m = mape(y_true, y_pred)
    s = smape(y_true, y_pred)
    p = precision_promedio(y_true, y_pred)
    p2 = precision_punto_a_punto(y_true, y_pred)
    da = directional_accuracy(y_true, y_pred)
    corr = pearson_safe(y_true, y_pred)

    print(f"\n=== {nombre} ===")
    print(f"MAE: {mae:.2f} USD")
    print(f"RMSE: {r:.2f} USD")
    print(f"MAPE: {m:.2f}%")
    print(f"sMAPE: {s:.2f}%")
    print(f"Precisión promedio: {p:.2f}%")
    print(f"Precisión punto a punto (media %): {p2:.2f}%")
    print(f"Directional Accuracy: {da if not np.isnan(da) else 'NA'}%")
    print(f"Correlación Pearson: {corr if not np.isnan(corr) else 'NA'}")
    return {
        "MAE": mae,
        "RMSE": r,
        "MAPE(%)": m,
        "sMAPE(%)": s,
        "Prec.(%)": p,
        "Prec Punto a Punto(%)": p2,
        "Directional Acc.(%)": da,
        "Pearson": corr
    }

# ------------------------------
# Ejecutar métricas
# ------------------------------

print("\n=== METRICAS — CASO 2024 (Promedio 7 días) ===")
met_lstm_2024 = resumen_metrics(y_test_2024_real, preds_2024_real, "LSTM 2024")
met_tf_2024 = resumen_metrics(y_test_2024_real, preds_2024_tf_real, "Transformer 2024")

print("\n=== METRICAS — CASO HISTÓRICO (Promedio 30 días) ===")
met_lstm_total = resumen_metrics(y_test_total_real, preds_total_real, "LSTM Histórico")
met_tf_total = resumen_metrics(y_test_total_real, preds_total_tf_real, "Transformer Histórico")

# ------------------------------
# Crear tabla consolidada y guardado
# ------------------------------

metrics_df = pd.DataFrame([
    {"Modelo": "LSTM 2024 (7 días)", **met_lstm_2024},
    {"Modelo": "Transformer 2024 (7 días)", **met_tf_2024},
    {"Modelo": "LSTM Histórico (30 días)", **met_lstm_total},
    {"Modelo": "Transformer Histórico (30 días)", **met_tf_total},
])

print("\n=== TABLA DE MÉTRICAS COMPLETAS ===\n")
print(metrics_df)
metrics_df.to_csv("metrics_resultados_completos.csv", index=False)
print("\nArchivo 'metrics_resultados_completos.csv' guardado.")

# ------------------------------
# Fechas reales para eje X (desde Bloque 1)
# ------------------------------

fechas_test_2024 = datos_2024[datos_2024["Split"] == "test_2024_3m"]["Date"].values
fechas_test_total = datos_total[datos_total["Split"] == "test_2024_full_history"]["Date"].values

# Alinear fechas con los arrays (recortar al mismo tamaño)
def align_dates_with_series(dates, series_len):
    dates = np.asarray(dates)
    if len(dates) > series_len:
        return dates[-series_len:]
    return dates

fechas_test_2024 = align_dates_with_series(fechas_test_2024, len(y_test_2024_real))
fechas_test_total = align_dates_with_series(fechas_test_total, len(y_test_total_real))

# ------------------------------
# Plots
# ------------------------------

plt.style.use("seaborn-v0_8-darkgrid")

def plot_series_dates(fechas, y_real, y_lstm, y_tf=None, title=""):
    plt.figure(figsize=(14,6))
    plt.plot(fechas, y_real, label="Real", linewidth=2, color="black")
    plt.plot(fechas, y_lstm, label="LSTM", linewidth=2)
    if y_tf is not None:
        plt.plot(fechas, y_tf, label="Transformer", linewidth=2)
    plt.title(title, fontsize=14)
    plt.xlabel("Fecha")
    plt.ylabel("Precio (USD)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_error_dates(fechas, error_lstm, error_tf=None, title=""):
    plt.figure(figsize=(14,4))
    plt.plot(fechas, error_lstm, label="Error LSTM")
    if error_tf is not None:
        plt.plot(fechas, error_tf, label="Error Transformer")
    plt.title(title, fontsize=14)
    plt.xlabel("Fecha")
    plt.ylabel("Error absoluto (USD)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# CASO 2024
plot_series_dates(
    fechas_test_2024,
    y_test_2024_real,
    preds_2024_real,
    preds_2024_tf_real,
    title="Predicción semanal (H=7) — Año 2024"
)

err_lstm_2024 = np.abs(y_test_2024_real - preds_2024_real)
err_tf_2024 = np.abs(y_test_2024_real - preds_2024_tf_real)

plot_error_dates(
    fechas_test_2024,
    err_lstm_2024,
    err_tf_2024,
    title="Error absoluto — Predicción semanal 2024"
)

# CASO HISTÓRICO
plot_series_dates(
    fechas_test_total,
    y_test_total_real,
    preds_total_real,
    preds_total_tf_real,
    title="Predicción mensual (H=30) — Histórico (2010–2024)"
)

err_lstm_total = np.abs(y_test_total_real - preds_total_real)
err_tf_total = np.abs(y_test_total_real - preds_total_tf_real)

plot_error_dates(
    fechas_test_total,
    err_lstm_total,
    err_tf_total,
    title="Error absoluto — Predicción mensual Histórico"
)